In [64]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 0.1
eval_iters = 250
n_embd = 384

cuda


In [3]:
# Opening the book 
with open('./books/wizard_of_oz.txt','r', encoding='utf-8') as f:
    text = f.read()

print(text[:200])

﻿                            THE MAGIC OF OZ


       A Faithful Record of the Remarkable Adventures of Dorothy
            and Trot and the Wizard of Oz, together with the
               Cowardly Lio


In [6]:
# Get Set of chars in the book
chars = sorted(set(text))
print(chars)
len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


79

In [8]:
#Assign a int to each char contained in the book, creating a simple char tokenizer as well as a encode and decode function
string_to_int = {ch: i for i,ch in enumerate(chars) }
int_to_string = {i: ch for i,ch in enumerate(chars) }

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

print(encode('hello'))
print(decode(encode('hello')))

[59, 56, 63, 63, 66]
hello


In [11]:
#Converting the text to ints and then the data to tensor
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])


tensor([78,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 43, 31, 28,  1, 36, 24, 30,
        32, 26,  1, 38, 29,  1, 38, 48,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,
        24,  1, 29, 52, 60, 71, 59, 57, 72, 63,  1, 41, 56, 54, 66, 69, 55,  1,
        66, 57,  1, 71, 59, 56,  1, 41, 56, 64, 52, 69, 62, 52, 53, 63, 56,  1,
        24, 55, 73, 56, 65, 71, 72, 69, 56, 70])


In [12]:
#Bagram Language Model
#Dividing data in train and test datasets
n = int(0.8*len(data))
train_data = data[:n]
test_data = data[n:]


In [18]:
#Defining the block size and passing through Train_data


x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} target is {target}")

when input is tensor([78]) target is 1
when input is tensor([78,  1]) target is 1
when input is tensor([78,  1,  1]) target is 1
when input is tensor([78,  1,  1,  1]) target is 1
when input is tensor([78,  1,  1,  1,  1]) target is 1
when input is tensor([78,  1,  1,  1,  1,  1]) target is 1
when input is tensor([78,  1,  1,  1,  1,  1,  1]) target is 1
when input is tensor([78,  1,  1,  1,  1,  1,  1,  1]) target is 1


In [22]:
#testing
randint = torch.randint(-100,100,(6,))
randint

tensor([ -1,  35, -28,   0,   1,  22])

In [27]:
# zeros, ones, empty, arange, linspace, logspace, eye, empty like

a = torch.empty((2,3), dtype=torch.int64)
print(a)
empty_like = torch.empty_like(a)
print(empty_like)

tensor([[37, 28, 32],
        [35, 35,  0]])
tensor([[1, 1, 1],
        [1, 1, 1]])


In [35]:
#Tracking Time
start_time = time.time()
zeros = torch.zeros(1,1)
end_time = time.time()

elapse_time = end_time -  start_time
print(f"Time: {elapse_time:.8f}")


Time: 0.00000000


In [34]:
# Why to use the GPU over CPU
torch_rand1 = torch.rand(100, 100, 100, 100).to(device)
torch_rand2 = torch.rand(100, 100, 100, 100).to(device)
np_rand1 = torch.rand(100, 100, 100, 100)
np_rand2 = torch.rand(100, 100, 100, 100)

start_time = time.time()

rand = (torch_rand1 @ torch_rand2)

end_time = time.time()

elapsed_time = end_time - start_time
print(f"GPU Time: {elapsed_time:.8f}")


start_time = time.time()

rand = np.multiply(np_rand1, np_rand2)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"CPU Time: {elapsed_time:.8f}")



GPU Time: 0.03091812
CPU Time: 0.11285615


In [36]:
# torch.stack, torch.multinominal, torch.tril, torch.triu, input.T / input.transpose, nn.Linear, torch.cat, F.softmax 

# Define a probability tensor
probabilities = torch.tensor([0.1, 0.9])
# 10% or 0.1 => 0, 90% or 0.9 => 1. each probability points to the index of the probability in the tensor
# Draw 5 samples from the multinomial distribution
samples = torch.multinomial(probabilities, num_samples=10, replacement=True)
print(samples)

tensor([1, 1, 1, 0, 0, 1, 1, 1, 1, 1])


In [38]:
out = torch.tril(torch.ones(5, 5))
out

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])

In [39]:
out = torch.triu(torch.ones(5, 5))
out

tensor([[1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1.]])

In [40]:
out = torch.zeros(5, 5).masked_fill(torch.tril(torch.ones(5, 5)) == 0, float('-inf'))
out


tensor([[0., -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0.]])

In [41]:
torch.exp(out)

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])

In [42]:
input = torch.zeros(2, 3, 4)
out1 = input.transpose(0, 1)
out2 = input.transpose(-2,-1)
print(out1.shape)
print(out2.shape)

torch.Size([3, 2, 4])
torch.Size([2, 4, 3])


In [43]:
tensor1 = torch.tensor([1, 2, 3])
tensor2 = torch.tensor([4, 5, 6])
tensor3 = torch.tensor([7, 8, 9])

# Stack the tensors along a new dimension
stacked_tensor = torch.stack([tensor1, tensor2, tensor3])
stacked_tensor

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])

In [44]:
import torch.nn as nn
sample = torch.tensor([10.,10.,10.])
linear = nn.Linear(3, 3, bias=False)
print(linear(sample))



tensor([ 3.9546, -1.5305,  3.6104], grad_fn=<SqueezeBackward4>)


In [45]:
import torch.nn.functional as F

# Create a tensor
tensor1 = torch.tensor([1.0, 2.0, 3.0])

# Apply softmax using torch.nn.functional.softmax()
softmax_output = F.softmax(tensor1, dim=0)

print(softmax_output)



tensor([0.0900, 0.2447, 0.6652])


In [47]:


# Initialize an embedding layer
vocab_size = 79
embedding_dim = 6
embedding = nn.Embedding(vocab_size, embedding_dim)

# Create some input indices
input_indices = torch.LongTensor([1, 5, 3, 2])

# Apply the embedding layer
embedded_output = embedding(input_indices)

# The output will be a tensor of shape (4, 100), where 4 is the number of inputs
# and 100 is the dimensionality of the embedding vectors
print(embedded_output.shape)
print(embedded_output)



torch.Size([4, 6])
tensor([[ 1.7222,  1.8800, -1.0589,  1.8347,  1.0325, -0.2926],
        [ 0.0022,  1.4499,  0.2764,  0.9480,  0.0027, -0.3228],
        [-0.7922,  0.1078,  0.1276,  2.1624, -0.5695, -1.7326],
        [ 0.5407, -0.8879, -0.0760, -0.2084,  0.4501,  1.2157]],
       grad_fn=<EmbeddingBackward0>)


In [48]:
a = torch.tensor([[1,2],[3,4],[5,6]])
b = torch.tensor([[7,8,9],[10,11,12]])
# print(a @ b)
print(torch.matmul(a, b))



tensor([[ 27,  30,  33],
        [ 61,  68,  75],
        [ 95, 106, 117]])


In [56]:
# Back to our Regular Program Schedule


def get_batch(split):
    data = train_data if split == 'train' else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x,y = get_batch('train')
print(x)
print(y)

tensor([[69, 52, 57, 71,  1, 60, 65, 71],
        [ 1, 60, 71,  1, 60, 65, 70, 60],
        [58, 59,  1, 52,  1, 57, 66, 69],
        [62, 60,  8,  1, 74, 59, 66,  1]], device='cuda:0')
tensor([[52, 57, 71,  1, 60, 65, 71, 66],
        [60, 71,  1, 60, 65, 70, 60, 55],
        [59,  1, 52,  1, 57, 66, 69, 56],
        [60,  8,  1, 74, 59, 66,  1, 69]], device='cuda:0')


In [58]:
#Creating the Model
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


__3 no)lR8MBUsR.I3k36zZ8&VtNq-qQf"dWOvmE4j_Hd(SMc 2g4b23;"h-zeWzQYZYZfRjr"HW9F3O)lJyf]N:,lj e0KA702x5Bozk'YMLvgHLdldg7Eoy8mG7hsVyTZfbA3DiIF(gpSO3mn"N4'Yt2iPc;R:Irmqmgei0WB mlF)zkP9FQGbT9eyaiGdsgj?qv_cT8zeA8PSOr81bLSV57B
C3s
Y!YczEQA8PN4h]"hxwgSPS lb.VsuUl
fh av(ljUR 
0DILbqj01603UiWlwZ[6
bMvHmnwn9﻿dxBEA3cJULd':(de4]Qi6audR0t9Td;IdVxn.DI9?,0FdTDQAC9?ZbYbLs)0Zd?:H(Sc8'n[P(SEmg0W2xM3HDqg_V.ho)(5Baj03;bTk4TV-_YbYcOOkP!Y9FdCC5U3Dxj;'Y﻿xEFN]D]jh,Z(zEfEf!YZ WOAhovvp&'ei"SJwe
do7G5Vj3ovj[P_W﻿rn]oIR.Fmvh


In [67]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [69]:


# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())



step: 0, train loss: 3.033, val loss: 3.061
step: 250, train loss: 3.017, val loss: 3.016
step: 500, train loss: 2.997, val loss: 3.010
step: 750, train loss: 2.978, val loss: 2.999
3.238788366317749


In [ ]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)